# CMPT 459: Project Milestone 3
# Group: Haiyaa


In [97]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
# from sklearn.tree import plot_tree
# from lightgbm import LGBMClassifier
import pandas as pd
import numpy as np
import pickle
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, log_loss
# multilabel_confusion_matrix
from sklearn.metrics import precision_score, recall_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from lightgbm import LGBMClassifier

from sklearn.metrics import make_scorer
from sklearn.metrics import recall_score, accuracy_score
from sklearn.model_selection import GridSearchCV

import random
random.seed(0)

In [129]:
BASE_URL = f'data'
base_data_filename = '1-5.csv'
base_data_path = f'{BASE_URL}/{base_data_filename}'
train_data_filename_x = 'x_train.csv' # your processed filename for x
train_data_path_x = f'{BASE_URL}/{train_data_filename_x}'
train_data_filename_y = 'y_train.csv' # your processed filename for y
train_data_path_y = f'{BASE_URL}/{train_data_filename_y}'
test_data_filename_x = 'x_test.csv' # your processed filename for x
test_data_path_x = f'{BASE_URL}/{test_data_filename_x}'
test_data_filename_y = 'y_test.csv' # your processed filename for y
test_data_path_y = f'{BASE_URL}/{test_data_filename_y}'

# Load Train and Test Data

In [53]:
### Started from what we have until Milestone Q1.5
data = pd.read_csv(base_data_path, index_col = False, keep_default_na=False)
# x_train = pd.read_csv(train_data_path_x, index_col = False, keep_default_na=False)
# y_train = pd.read_csv(train_data_path_y, index_col = False, keep_default_na=False)
# x_test = pd.read_csv(test_data_path_x, index_col = False, keep_default_na=False)
# y_test = pd.read_csv(test_data_path_y, index_col = False, keep_default_na=False)

# Additional Data Pre-processing and Transformation

In [62]:
idx = np.where(data['date_confirmation'].str.len() > 10)[0]
data.loc[idx, 'date_confirmation'] = data.loc[idx, 'date_confirmation'].str.split('-', expand=True)[0]

idx = np.where(data['date_confirmation']=='')[0]
data.loc[idx, 'date_confirmation'] = data.loc[idx, 'date_confirmation'].str.replace('', '0.0.0')

data[['day', 'month', 'year']] = data['date_confirmation'].str.split('.', expand=True)
data[['day', 'month', 'year']] = data[['day', 'month', 'year']].astype(int)
data.drop(columns='date_confirmation', inplace=True)

In [63]:
def convertText(x):
    if x == '':
        return 0
    else:
        return 1
    
data['source'] = data['source'].apply(convertText)
data['additional_information'] = data['additional_information'].apply(convertText)

In [64]:
sex_info = pd.get_dummies(data['sex'])
address_info = pd.get_dummies(data[['province', 'country']])
data = pd.concat([data, sex_info, address_info], axis=1)
data.drop(columns=['sex', 'province', 'country', 'age_min', 'age_max'], inplace=True)
### Drop the index column
data.drop(columns = 'Unnamed: 0', inplace=True)

# Split Data

In [65]:
feature_labels = list(data)
feature_labels.remove('outcome')

In [66]:
X_values = data[feature_labels]
y_values = data.outcome

In [67]:
x_train, x_test, y_train, y_test = train_test_split(X_values, y_values, test_size=0.25)

In [130]:
## Save the files
x_train.to_csv(train_data_path_x, index=False)
x_test.to_csv(test_data_path_x, index=False)
y_train.to_csv(train_data_path_y, index=False)
y_test.to_csv(test_data_path_y, index=False)

# Define Score Fcuntion

In [98]:
def get_recall_on_deseased(y_true, y_pred):
    ret = precision_recall_fscore_support(y_true, y_pred, average = None, labels=['deceased', 'hospitalized', 'nonhospitalized', 'recovered'])
    return ret[1][0]

def get_overall_recall(y_true, y_pred):
#     ret = precision_recall_fscore_support(y_true, y_pred, average = None, labels=['deceased', 'hospitalized', 'nonhospitalized', 'recovered'])
#     return ret[1].mean()
    ret = recall_score(y_true, y_pred, average='macro')
    return ret


# Decision Tree

In [99]:
dt = DecisionTreeClassifier()
# dt_paras = {'criterion': ('gini', 'entropy'), 'max_depth':[5, 10], 'min_samples_split': [2, 3]}
# dt_paras = {'criterion': ('gini', 'entropy'), 'min_samples_leaf': [1, 2, 3]}
dt_paras = {'criterion': ('gini', 'entropy'), 'min_samples_leaf': [1, 2, 3], 'max_features': [0.8, 0.6, 0.4]}
dt_scoring = {'recall_deseased': make_scorer(get_recall_on_deseased), 'recall': make_scorer(get_overall_recall), 'accuracy': make_scorer(accuracy_score)}
dt_cv = 5
dt_grid_search = GridSearchCV(dt, param_grid=dt_paras, scoring=dt_scoring, cv=dt_cv, n_jobs=-1, refit='recall_deseased')
dt_grid_search.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ('gini', 'entropy'),
                         'max_features': [0.8, 0.6, 0.4],
                         'min_samples_leaf': [1, 2, 3]},
             refit='recall_deseased',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'recall': make_scorer(get_overall_recall),
                      'recall_deseased': make_scorer(get_recall_on_deseased)})

In [124]:
pd.DataFrame(dt_grid_search.cv_results_)[['param_criterion', 'param_max_features', 'param_min_samples_leaf', 'mean_test_recall_deseased', 'rank_test_recall_deseased', 'mean_test_recall', 'rank_test_recall', 'mean_test_accuracy', 'rank_test_accuracy']]

,param_criterion,param_max_features,param_min_samples_leaf,mean_test_recall_deseased,rank_test_recall_deseased,mean_test_recall,rank_test_recall,mean_test_accuracy,rank_test_accuracy
0,gini,0.8,1,0.120104,3,0.665724,4,0.880263,5
1,gini,0.8,2,0.112533,10,0.662421,9,0.878830,10
2,gini,0.8,3,0.105222,16,0.660407,14,0.878902,9
3,gini,0.6,1,0.120627,1,0.665760,2,0.880170,6
4,gini,0.6,2,0.114621,7,0.662588,8,0.878711,12
5,gini,0.6,3,0.107572,13,0.660269,15,0.878039,15
6,gini,0.4,1,0.119843,5,0.665755,3,0.880399,1
7,gini,0.4,2,0.112794,9,0.662000,11,0.878415,13
8,gini,0.4,3,0.101828,18,0.658023,18,0.877321,17
9,entropy,0.8,1,0.120366,2,0.665789,1,0.880315,2


In [136]:
pd.DataFrame(dt_grid_search.cv_results_).to_csv('decision-tree_results.csv', index=False)

In [131]:
dt_grid_search.best_estimator_

DecisionTreeClassifier(max_features=0.6)

In [133]:
best_dt = dt_grid_search.best_estimator_
print('mean cv score: ', dt_grid_search.best_score_)
print('accuracy: ', accuracy_score(y_test, best_dt.predict(x_test)))
print('overall recall: ', recall_score(y_test, best_dt.predict(x_test), average='macro'))
print(precision_recall_fscore_support(y_train, best_dt.predict(x_train), average = None, labels=['deceased', 'hospitalized', 'nonhospitalized', 'recovered']))
print(precision_recall_fscore_support(y_test, best_dt.predict(x_test), average = None, labels=['deceased', 'hospitalized', 'nonhospitalized', 'recovered']))

mean cv score:  0.1206266318537859
accuracy:  0.8829942491790747
overall recall:  0.6727089911179521
(array([0.84579977, 0.83288934, 0.99995943, 0.81324403]), array([0.19190601, 0.93334281, 0.99996754, 0.64198441]), array([0.31283252, 0.88025946, 0.99996348, 0.71753681]), array([  3830, 147726, 123234,  70550]))
(array([0.54603175, 0.82768995, 0.99956381, 0.79357945]), array([0.13469068, 0.92416222, 0.99951536, 0.6324677 ]), array([0.2160804 , 0.87326978, 0.99953958, 0.70392255]), array([ 1277, 49118, 41268, 23451]))


# Random Forest

In [107]:
rf = RandomForestClassifier()
rf_paras = {'criterion': ('gini', 'entropy'), 'n_estimators': [50, 100, 200], 'max_features': [0.8, 0.6, 0.4]}
rf_scoring = {'recall_deseased': make_scorer(get_recall_on_deseased), 'recall': make_scorer(get_overall_recall), 'accuracy': make_scorer(accuracy_score)}
rf_cv = 5
rf_grid_search = GridSearchCV(rf, param_grid=rf_paras, scoring=rf_scoring, cv=rf_cv, n_jobs=-1, refit='recall_deseased')
rf_grid_search.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ('gini', 'entropy'),
                         'max_features': [0.8, 0.6, 0.4],
                         'n_estimators': [50, 100, 200]},
             refit='recall_deseased',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'recall': make_scorer(get_overall_recall),
                      'recall_deseased': make_scorer(get_recall_on_deseased)})

In [126]:
pd.DataFrame(rf_grid_search.cv_results_)[['param_criterion', 'param_n_estimators', 'param_max_features', 'mean_test_recall_deseased', 'rank_test_recall_deseased', 'mean_test_recall', 'rank_test_recall', 'mean_test_accuracy', 'rank_test_accuracy']]

,param_criterion,param_n_estimators,param_max_features,mean_test_recall_deseased,rank_test_recall_deseased,mean_test_recall,rank_test_recall,mean_test_accuracy,rank_test_accuracy
0,gini,50,0.8,0.115144,8,0.666001,7,0.881256,15
1,gini,100,0.8,0.115144,10,0.665852,12,0.881068,18
2,gini,200,0.8,0.115405,7,0.665931,10,0.881291,13
3,gini,50,0.6,0.114099,17,0.665668,18,0.881233,17
4,gini,100,0.6,0.113577,18,0.665816,14,0.881398,3
5,gini,200,0.6,0.114883,11,0.665904,11,0.881308,10
6,gini,50,0.4,0.115144,8,0.666129,5,0.881253,16
7,gini,100,0.4,0.114883,11,0.665958,8,0.881297,12
8,gini,200,0.4,0.114621,14,0.665755,15,0.881375,6
9,entropy,50,0.8,0.115927,2,0.666243,2,0.881430,2


In [137]:
pd.DataFrame(rf_grid_search.cv_results_).to_csv('random-forest_results.csv', index=False)

In [112]:
rf_grid_search.best_estimator_

RandomForestClassifier(criterion='entropy', max_features=0.6, n_estimators=50)

In [134]:
best_rf = rf_grid_search.best_estimator_
print('mean cv score: ', rf_grid_search.best_score_)
print('accuracy: ', accuracy_score(y_test, best_rf.predict(x_test)))
print('overall recall: ', recall_score(y_test, best_rf.predict(x_test), average='macro'))
print(precision_recall_fscore_support(y_train, best_rf.predict(x_train), average = None, labels=['deceased', 'hospitalized', 'nonhospitalized', 'recovered']))
print(precision_recall_fscore_support(y_test, best_rf.predict(x_test), average = None, labels=['deceased', 'hospitalized', 'nonhospitalized', 'recovered']))

mean cv score:  0.11671018276762402
accuracy:  0.8837760828396198
overall recall:  0.6726402967525938
(array([0.93830703, 0.83439831, 0.99995943, 0.8079131 ]), array([0.17075718, 0.93079756, 0.99996754, 0.64833451]), array([0.28893307, 0.8799657 , 0.99996348, 0.71938033]), array([  3830, 147726, 123234,  70550]))
(array([0.72368421, 0.82952175, 0.99953965, 0.79008177]), array([0.12920908, 0.92308319, 0.99966075, 0.63860816]), array([0.2192691 , 0.87380512, 0.9996002 , 0.70631514]), array([ 1277, 49118, 41268, 23451]))


# XGB

In [ ]:
xgb = XGBClassifier()
xgb_paras = {'n_estimators': [50, 100], 'booster': ['gbtree', 'gblinear', 'dart'], 'max_depth': [5, 10, 15]}
# xgb_paras = {'booster': ['gbtree', 'gblinear', 'dart']}
xgb_scoring = {'recall_deseased': make_scorer(get_recall_on_deseased), 'recall': make_scorer(get_overall_recall), 'accuracy': make_scorer(accuracy_score)}
xgb_cv = 5
xgb_grid_search = GridSearchCV(xgb, param_grid=xgb_paras, scoring=xgb_scoring, cv=xgb_cv, refit='recall_deseased')
xgb_grid_search.fit(x_train, y_train)

In [ ]:
pd.DataFrame(xgb_grid_search.cv_results_)

In [ ]:
xgb_grid_search.best_estimator_

In [ ]:
best_xgb = xgb_grid_search.best_estimator_
print('mean cv score: ', xgb_grid_search.best_score_)
print('accuracy: ', accuracy_score(y_test, best_xgb.predict(x_test)))
print('overall recall: ', recall_score(y_test, best_xgb.predict(x_test), average='macro'))
print(precision_recall_fscore_support(y_train, best_xgb.predict(x_train), average = None, labels=['deceased', 'hospitalized', 'nonhospitalized', 'recovered']))
print(precision_recall_fscore_support(y_test, best_xgb.predict(x_test), average = None, labels=['deceased', 'hospitalized', 'nonhospitalized', 'recovered']))

# LGB

In [103]:
lgb = LGBMClassifier(class_weight='balanced')
lgb_paras = {'n_estimators': [50, 100, 200], 'boosting_type': ['gbdt', 'dart', 'goss'], 'num_leaves': [21, 31, 41]}
# lgb_paras = {'boosting_type': ['gbdt', 'dart', 'goss']}
lgb_scoring = {'recall_deseased': make_scorer(get_recall_on_deseased), 'recall': make_scorer(get_overall_recall), 'accuracy': make_scorer(accuracy_score)}
lgb_cv = 5
lgb_grid_search = GridSearchCV(lgb, param_grid=lgb_paras, scoring=lgb_scoring, cv=lgb_cv, n_jobs=-1, refit='recall_deseased')
lgb_grid_search.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=LGBMClassifier(class_weight='balanced'), n_jobs=-1,
             param_grid={'boosting_type': ['gbdt', 'dart', 'goss'],
                         'n_estimators': [50, 100, 200],
                         'num_leaves': [21, 31, 41]},
             refit='recall_deseased',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'recall': make_scorer(get_overall_recall),
                      'recall_deseased': make_scorer(get_recall_on_deseased)})

In [132]:
pd.DataFrame(lgb_grid_search.cv_results_)[['param_n_estimators', 'param_boosting_type', 'param_num_leaves', 'mean_test_recall_deseased', 'rank_test_recall_deseased', 'mean_test_recall', 'rank_test_recall', 'mean_test_accuracy', 'rank_test_accuracy']]

,param_n_estimators,param_boosting_type,param_num_leaves,mean_test_recall_deseased,rank_test_recall_deseased,mean_test_recall,rank_test_recall,mean_test_accuracy,rank_test_accuracy
0,50,gbdt,21,0.695039,11,0.694783,24,0.687986,24
1,50,gbdt,31,0.697128,8,0.703992,18,0.700611,19
2,50,gbdt,41,0.688773,16,0.709601,13,0.711568,14
3,100,gbdt,21,0.696867,9,0.708643,15,0.707407,15
4,100,gbdt,31,0.691384,15,0.713858,8,0.717869,10
5,100,gbdt,41,0.664491,22,0.717438,4,0.730741,5
6,200,gbdt,21,0.675718,20,0.717074,5,0.727984,6
7,200,gbdt,31,0.669452,21,0.721963,2,0.736439,2
8,200,gbdt,41,0.653786,24,0.724513,1,0.745468,1
9,50,dart,21,0.721410,2,0.688068,26,0.667531,27


In [138]:
pd.DataFrame(lgb_grid_search.cv_results_).to_csv('lgb_results.csv', index=False)

In [105]:
lgb_grid_search.best_estimator_

LGBMClassifier(boosting_type='dart', class_weight='balanced', n_estimators=200,
               num_leaves=21)

In [135]:
best_lgb = lgb_grid_search.best_estimator_
print('mean cv score: ', lgb_grid_search.best_score_)
print('accuracy: ', accuracy_score(y_test, best_lgb.predict(x_test)))
print('overall recall: ', recall_score(y_test, best_lgb.predict(x_test), average='macro'))
print(precision_recall_fscore_support(y_train, best_lgb.predict(x_train), average = None, labels=['deceased', 'hospitalized', 'nonhospitalized', 'recovered']))
print(precision_recall_fscore_support(y_test, best_lgb.predict(x_test), average = None, labels=['deceased', 'hospitalized', 'nonhospitalized', 'recovered']))

mean cv score:  0.7229765013054831
accuracy:  0.6922615841687371
overall recall:  0.7085587914677808
(array([0.03857026, 0.92083514, 0.9995617 , 0.62119031]), array([0.77310705, 0.46007473, 0.99930214, 0.63009213]), array([0.07347486, 0.61358527, 0.9994319 , 0.62560956]), array([  3830, 147726, 123234,  70550]))
(array([0.03730872, 0.91773279, 0.99936993, 0.62354474]), array([0.74079875, 0.46150088, 0.99929728, 0.63263827]), array([0.07103969, 0.61415915, 0.9993336 , 0.62805859]), array([ 1277, 49118, 41268, 23451]))
